## **Generamos un conjunto de tamaño similar al nuestro para probar el tiempo de entrenamiento**

In [1]:
import pandas as pd
import joblib
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

path_emb6= "/content/drive/MyDrive/TFM/embedding_dnabert6.csv"
emb6= pd.read_csv(path_emb6)
emb6 = emb6.drop(columns=['Unnamed: 0'])
embedding_columns6 = [col for col in emb6.columns if col != 'gene']

# Función para concatenar los embeddings en un solo vector
def concatenate_values(row):
    return row.values[1:].tolist()  # Ignorar la primera columna (filename) y convertir a lista

# Aplicar la función a cada fila para concatenar embeddings en un solo vector
emb6['concatenated_vector'] = emb6.apply(concatenate_values, axis=1)
emb6.drop(columns=embedding_columns6, inplace=True)

Mounted at /content/drive


In [2]:
path_int= "/content/drive/MyDrive/genes.csv"
inter = pd.read_csv(path_int)
inter.head(5)
inter['interactant_id'] = inter['interactant_id'].astype(str)
inter['gene_id'] = inter['gene_id'].astype(str)
emb6['gene'] = emb6['gene'].astype(str)

In [3]:
#info gen 1
inter_emb = pd.merge(inter, emb6, left_on='gene_id', right_on='gene', how='left')
inter_emb.rename(columns={'concatenated_vector': 'gen_vector'}, inplace=True)
#info gen 2
inter_emb = pd.merge(inter_emb, emb6, left_on='interactant_id', right_on='gene', how='left')
inter_emb.rename(columns={'concatenated_vector': 'inter_vector'}, inplace=True)

inter_emb.drop(columns=['gene_x'], inplace=True)
inter_emb.drop(columns=['gene_y'], inplace=True)

inter_emb = inter_emb[['gene_id', 'gen_vector','Symbol_x','seqname_x', 'interactant_id','inter_vector','Symbol_y','seqname_y']]

In [4]:
inter_emb['gen_vector'] = inter_emb['gen_vector'].apply(lambda x: np.array(x))
inter_emb['inter_vector'] = inter_emb['inter_vector'].apply(lambda x: np.array(x))

# Multiplicar los arrays
inter_emb['result'] = inter_emb.apply(lambda row: row['gen_vector'] * row['inter_vector'], axis=1)

In [5]:
import random

X_train, X_test = train_test_split(inter_emb, test_size=0.1, random_state=42)
df_splits = np.array_split(X_train, 100)
# Selecciona 10 muestras aleatorias de las 100 divisiones
random_splits = random.sample(df_splits, 10)

In [6]:
# Since visualizing high-dimensional data directly is not feasible,
# the plot_decision_boundary function will not be used in this context.

# Train OSVM models on each subset and collect support vectors
support_vectors = np.vstack([
    OneClassSVM(kernel='rbf', gamma='auto', nu=0.1).fit(np.vstack(df_split['result'].values)).support_vectors_
    for df_split in random_splits
])

# Train a final OSVM model on the collected support vectors
final_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
final_model.fit(support_vectors)

# Note: Visualization of decision boundaries in high-dimensional space is not straightforward
# and typically not practical. You might consider dimensionality reduction techniques
# like PCA for visualization purposes, but it's outside the direct scope of this modification.

OneClassSVM(gamma='auto', nu=0.1)

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Función para calcular y mostrar métricas
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    print(f"Precisión: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Exactitud: {accuracy:.4f}")
    print("-----------------------------")


In [8]:
X_test.shape

(83851, 9)

In [9]:
test_data =  np.vstack(X_test['result'].values)
y_true = np.ones(83851)

preds = final_model.predict(test_data)
calculate_metrics(y_true, preds)

Precisión: 1.0000
Recall: 0.9899
F1-Score: 0.9949
Exactitud: 0.9899
-----------------------------


In [10]:
X_test['prediccion'] = preds

In [11]:
X_test.groupby('prediccion').size()

prediccion
-1      843
 1    83008
dtype: int64

# Probamos con interacciones negativas

In [12]:
path_int_neg_OC = '/content/drive/MyDrive/inter_neg_OCSVM.csv'
inter_neg_modelo = pd.read_csv(path_int_neg_OC)

In [13]:
inter_neg_modelo = inter_neg_modelo.dropna()

In [14]:
inter_neg_modelo['interactant_id'] = inter_neg_modelo['interactant_id'].astype(int)
inter_neg_modelo['gene_id'] = inter_neg_modelo['gene_id'].astype(int)

<ipython-input-14-90a033360045>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_neg_modelo['interactant_id'] = inter_neg_modelo['interactant_id'].astype(int)
<ipython-input-14-90a033360045>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_neg_modelo['gene_id'] = inter_neg_modelo['gene_id'].astype(int)


In [15]:
inter_neg_modelo['interactant_id'] = inter_neg_modelo['interactant_id'].astype(str)
inter_neg_modelo['gene_id'] = inter_neg_modelo['gene_id'].astype(str)

In [16]:
#info gen 1
inter_embOC_neg = pd.merge(inter_neg_modelo, emb6, left_on='gene_id', right_on='gene', how='left')
inter_embOC_neg.rename(columns={'concatenated_vector': 'gen_vector'}, inplace=True)
#info gen 2
inter_embOC_neg = pd.merge(inter_embOC_neg, emb6, left_on='interactant_id', right_on='gene', how='left')
inter_embOC_neg.rename(columns={'concatenated_vector': 'inter_vector'}, inplace=True)

inter_embOC_neg.drop(columns=['gene_x'], inplace=True)
inter_embOC_neg.drop(columns=['gene_y'], inplace=True)

In [17]:
inter_embOC_neg['gen_vector'] = inter_embOC_neg['gen_vector'].apply(lambda x: np.array(x))
inter_embOC_neg['inter_vector'] = inter_embOC_neg['inter_vector'].apply(lambda x: np.array(x))

# Multiplicar los arrays
inter_embOC_neg['result'] = inter_embOC_neg.apply(lambda row: row['gen_vector'] * row['inter_vector'], axis=1)

In [18]:
df_unir = inter_emb[['gene_id', 'interactant_id', 'result']]
df_unir['clas'] = 1

df_unir_neg = inter_embOC_neg[['gene_id', 'interactant_id', 'result']]
df_unir_neg['clas'] = -1

<ipython-input-18-0594e927daf6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unir['clas'] = 1
<ipython-input-18-0594e927daf6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unir_neg['clas'] = -1


In [19]:
pred_data =  np.vstack(inter_embOC_neg['result'].values)
preds = final_model.predict(pred_data)

inter_embOC_neg['preds6'] = preds

In [20]:
inter_embOC_neg.groupby('preds6').size()

preds6
-1     14968
 1    761194
dtype: int64

# Cargamos los embeddings a las interacciones que vamos a meter al modelo

In [21]:
path_int_OC = '/content/drive/MyDrive/inter_OCSVM.csv'
inter_modelo = pd.read_csv(path_int_OC)

In [22]:
inter_modelo['interactant_id'] = inter_modelo['interactant_id'].astype(str)
inter_modelo['gene_id'] = inter_modelo['gene_id'].astype(str)

In [23]:
#info gen 1
inter_embOC = pd.merge(inter_modelo, emb6, left_on='gene_id', right_on='gene', how='left')
inter_embOC.rename(columns={'concatenated_vector': 'gen_vector'}, inplace=True)
#info gen 2
inter_embOC = pd.merge(inter_embOC, emb6, left_on='interactant_id', right_on='gene', how='left')
inter_embOC.rename(columns={'concatenated_vector': 'inter_vector'}, inplace=True)

inter_embOC.drop(columns=['gene_x'], inplace=True)
inter_embOC.drop(columns=['gene_y'], inplace=True)

In [24]:
inter_embOC['gen_vector'] = inter_embOC['gen_vector'].apply(lambda x: np.array(x))
inter_embOC['inter_vector'] = inter_embOC['inter_vector'].apply(lambda x: np.array(x))

# Multiplicar los arrays
inter_embOC['result'] = inter_embOC.apply(lambda row: row['gen_vector'] * row['inter_vector'], axis=1)

In [25]:
pred_data =  np.vstack(inter_embOC['result'].values)
preds = final_model.predict(pred_data)

In [26]:
inter_embOC['preds6'] = preds
inter_embOC.groupby('preds6').size()

preds6
-1     1028
 1    48131
dtype: int64

# Hacemos la regresión logística

In [27]:
import pandas as pd

# Asegúrate de que los DataFrames tengan las mismas columnas
df_unir = inter_emb[['gene_id', 'interactant_id', 'result']].copy()
df_unir['clas'] = 1

df_unir_neg = inter_embOC_neg[['gene_id', 'interactant_id', 'result']].copy()
df_unir_neg['clas'] = -1

# Concatenar los DataFrames
df_concatenado = pd.concat([df_unir, df_unir_neg])

# Resetear el índice si es necesario
df_concatenado.reset_index(drop=True, inplace=True)

df_concatenado


,gene_id,interactant_id,result,clas
0,1,2886,"[0.012644644641475623, 0.07812840681132194, 0....",1
1,1,80854,"[0.019459022941786572, 0.14190614839113544, 0....",1
2,1,84236,"[0.01677515949061269, 0.14738198531107383, 0.0...",1
3,1,148581,"[0.018877623045719705, 0.15002753371341981, 0....",1
4,1,5655,"[0.016275818404831267, 0.10126798943409553, 0....",1
...,...,...,...,...
1614660,7756,57476,"[0.018341078117444443, 0.24240436120750883, 0....",-1
1614661,4968,55504,"[0.017185767018728086, 0.1499822087692948, 0.0...",-1
1614662,1036,151742,"[0.023438658480974688, 0.2888823605511206, 0.0...",-1
1614663,1002,64147,"[0.016168901186748956, 0.1114667131119571, 0.0...",-1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Asumimos que df_concatenado ya está creado y contiene las columnas 'result' y 'clas'

# Separar características (X) y etiquetas (y)
X = np.vstack(df_concatenado['result'])
y = df_concatenado['clas']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo de regresión logística con más iteraciones y un solver diferente
model = LogisticRegression(solver='saga', max_iter=2000)

# Entrenar el modelo
model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy: 0.665671623303833
Classification Report:
               precision    recall  f1-score   support

          -1       0.66      0.62      0.64     77689
           1       0.67      0.71      0.69     83778

    accuracy                           0.67    161467
   macro avg       0.67      0.66      0.66    161467
weighted avg       0.67      0.67      0.66    161467

Confusion Matrix:
 [[48120 29569]
 [24414 59364]]


In [ ]:
pred_scaled = scaler.transform(np.vstack(inter_embOC['result'].values))
y_predscaled = model.predict(pred_scaled)
inter_embOC['preds6_logis'] = y_predscaled
inter_embOC.groupby('preds6_logis').size()

inter_embOC[['Name_x', 'Name_y', 'gene_id', 'interactant_id', 'seqname_x','seqname_y','preds6','preds6_logis']].to_csv('/content/drive/MyDrive/prediccion_6.csv', index = False)

In [ ]:
import joblib

# Guardar el modelo entrenado en un archivo
model_path = '/content/drive/MyDrive/logistic_regression_model_6.joblib'
joblib.dump(model, model_path)

print(f"Modelo guardado en {model_path}")

Modelo guardado en /content/drive/MyDrive/logistic_regression_model_6.joblib


# Calculamos la sensitividad

In [ ]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler

# Cargar el modelo desde el archivo
model_path = '/content/drive/MyDrive/logistic_regression_model_6.joblib'
model6 = joblib.load(model_path)

# Asumimos que df_concatenado ya está creado y contiene las columnas 'result' y 'clas'

# Separar características (X) y etiquetas (y)
X = np.vstack(df_concatenado['result'])
y = df_concatenado['clas']

# Dividir los datos en conjuntos de entrenamiento y prueba, obteniendo los índices
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(X, y, df_concatenado.index, test_size=0.1, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hacer predicciones en el conjunto de prueba
y_pred = model6.predict(X_test_scaled)

# Calcular la sensitividad para la clase 1
sensitivity = recall_score(y_test, y_pred, pos_label=1)
print("Sensitividad para la clase 1:", sensitivity)

# Crear un DataFrame con la parte de test de df_concatenado
df_test = df_concatenado.loc[test_indices].copy()
df_test['Predicted'] = y_pred

# Guardar el DataFrame en un archivo CSV si lo deseas
df_test[['gene_id','interactant_id','clas','Predicted']].to_csv('/content/drive/MyDrive/test_predictions_with_df_concatenado6.csv', index=False)


Sensitividad para la clase 1: 0.708586979875385


# Entrenamos el modelo solo con las características con los coeficientes más altos

In [28]:
t6 = [250, 5, 199, 332, 598, 681, 407, 365, 351, 394, 364, 393, 685,
       418, 767, 634, 711, 450, 242, 275, 614, 515, 694, 287, 571,  88,
       308, 616, 459, 521, 273, 518, 319, 180, 279, 331,  82, 750, 264,
       288, 167, 221, 575, 254,  64, 126, 449, 596, 404, 670,  29, 158,
       533, 415, 659, 617, 553, 223, 628, 397, 337, 274, 416, 259,  37,
       667, 195, 149, 536, 625, 699, 534, 692, 729, 720, 373, 121, 679,
       253, 516, 456,  31, 330, 122, 506, 329, 577, 645, 508,  86, 747,
       342, 671, 248, 539, 102, 570, 344, 362, 312]

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Asumimos que df_concatenado ya está creado y contiene las columnas 'result' y 'clas'

# Separar características (X) y etiquetas (y)
X = np.vstack(df_concatenado['result'])
y = df_concatenado['clas']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(X, y, df_concatenado.index, test_size=0.1, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_filtered = X_train_scaled[:, t6]
X_test_filtered = X_test_scaled[:, t6]

# Crear el modelo de regresión logística con más iteraciones y un solver diferente
model = LogisticRegression(solver='saga', max_iter=2000)

# Entrenar el modelo
model.fit(X_train_filtered, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_filtered)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Crear un DataFrame con la parte de test de df_concatenado
df_test = df_concatenado.loc[test_indices].copy()
df_test['Predicted'] = y_pred

# Guardar el DataFrame en un archivo CSV si lo deseas
df_test[['gene_id','interactant_id','clas','Predicted']].to_csv('/content/drive/MyDrive/test_predictions_with_df_concatenado6_nuevo.csv', index=False)

Accuracy: 0.6411155220571386
Classification Report:
               precision    recall  f1-score   support

          -1       0.64      0.58      0.61     77689
           1       0.64      0.70      0.67     83778

    accuracy                           0.64    161467
   macro avg       0.64      0.64      0.64    161467
weighted avg       0.64      0.64      0.64    161467

Confusion Matrix:
 [[44933 32756]
 [25192 58586]]


In [ ]:
import joblib

# Guardar el modelo entrenado en un archivo
model_path = '/content/drive/MyDrive/logistic_regression_model_6_nuevo100.joblib'
joblib.dump(model, model_path)

print(f"Modelo guardado en {model_path}")

Modelo guardado en /content/drive/MyDrive/logistic_regression_model_6_nuevo100.joblib
